In [ ]:
# President: 2016 (Trump), 2020 (Biden), 2024 (Trump)
# Governor: 2018 (Whitmer), 2022 (Whitmer)
# Secretary of State: 2018 (Benson), 2022 (Benson)
# Attorney General: 2018 (Nessel), 2022 (Nessel)
# U.S. Senate: 2014 (Peters), 2018 (Stabenow), 2020 (Peters), 2024 (Slotkin)
# U.S. House: every cycle
# State Senate: 2014, 2018, 2022
# State House: every cycle

# OFFICES = ['U.S. House', 'State House']
# YEARS = ['2022', '2024']

# OFFICES = ['U.S. Senate']
# YEARS = ['2020', '2024']

# OFFICES = ['State Senate']
# YEARS = ['2022']

# OFFICES = ['President']
# YEARS = ['2024'] # you can only do 2024, given only two previous elections

# Not enough data
# # OFFICES = ['Governor', 'Secretary of State', 'Attorney General']
# # YEARS = ['2018', '2022']

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from functools import reduce
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LassoCV, LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import csv
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import xgboost as xgb

In [ ]:
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

### Compute partisanship

In [ ]:
def categorize_partisanship(row):
    if row["dem_share_prev"] >= 0.667:
        return "strong democrat"
    elif row["dem_share_prev"] >= 0.501:
        return "leans democrat"
    elif row["rep_share_prev"] >= 0.667:
        return "strong republican"
    elif row["rep_share_prev"] >= 0.501:
        return "leans republican"
    elif row["oth_share_prev"] >= 0.667:
        return "strong independent"
    elif row["oth_share_prev"] >= 0.501:
        return "leans independent"
    else:
        return "neutral"

In [ ]:
def categorize_partisan_change(row):
    # Model with a number line, ignore "other" parties
    # negative = left = more dem, positive = right = more rep
    change = row["rep_share_change"] - row["dem_share_change"]
    
    if np.abs(change) >= 0.01:
        if change > 0.5:
            return "gargantuanly more republican"
        if change > 0.35:
            return "massively more republican"
        if change > 0.25:
            return "much much more republican"
        if change > 0.15:
            return "much more republican"
        if change > 0.1:
            return "more republican"
        if change > 0.05:
            return "slightly more republican"
        elif change > 0.01:
            return "very slightly more republican"
        elif change > 0.005:
            return "infinitesimally more republican"
        elif change < -0.5:
            return "gargantuanly more democrat"
        elif change < -0.35:
            return "massively more democrat"
        elif change < -0.25:
            return "much much democrat"
        elif change < -0.15:
            return "much more democrat"
        elif change < -0.1:
            return "more democrat"
        elif change < -0.05:
            return "slightly more democrat"
        elif change < -0.01:
            return "very slightly more democrat"
        elif change < -0.005:
            return "infinitesimally more democrat"
    else:
        return "no change"

In [ ]:
def categorize_partisan_change_amount(row):
    # Model with a number line, ignore "other" parties
    # negative = left = more dem, positive = right = more rep
    change = row["rep_share_change"] - row["dem_share_change"]
    return change

### Predictions
US House and State House for 2018, 2020, 2022 ~ 45 mins

In [ ]:
census_datasets = [
    'b02001_race', 'b04007_ancestry', 'b05012_nativity_us', 'b08303_travel_time_work', 
    'b25003_housing_rentership', 'dp04_housing_characteristics', 'dp05_age_race', 's0101_age_sex', 
    's1101_households_families', 's1201_marital_status', 's1501_educational_attainment', 's1701_income_poverty', 
    's1903_median_income', 's2101_veteran_status', 's2201_food_stamps', 's2301_employment_status', 
    's2401_occupation_sex', 's2403_industry_sex', 's2501_occupancy_characteristics', 
    's2503_financial_characteristics', 's2701_health_insurance',
]

active_target = 'partisanship_change'
all_targets = ['partisanship', 'partisanship_change', 'partisanship_change_amount', 'turnout_pct']

# Rank n top features: [1, < infinity]
top_n_features_to_rank = 100000000

# Predict with n top features
# 16 = 0.4697 w/o engineered features
# 23 = 16 + 7 engineered fields we later throw away
top_n_feature_for_preds = 23

In [ ]:
for year in YEARS:
    print(f'Processing year {year}')
    
    for office in OFFICES:
        print(f'Processing office {office}')
        
        df_precincts = pd.read_csv('data/generated_data/df_06_tract_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.csv')
        df_precincts['standardized_id_num'] = df_precincts['standardized_id_num'].astype(str).str.zfill(13)
        
        df_precincts["partisanship"] = df_precincts.apply(categorize_partisanship, axis=1)
        df_precincts["partisanship_change"] = df_precincts.apply(lambda row: categorize_partisan_change(row), axis=1) # find the change
        df_precincts["partisanship_change_amount"] = df_precincts.apply(lambda row: categorize_partisan_change_amount(row), axis=1) # measure how big the change was

        print(f'Loading census data')
        census_dataset_dfs = []
        for census_dataset in census_datasets:
            census_dataset = census_dataset.lower()
            if census_dataset[:1] == 's':
                census_dataset_code = census_dataset[:5].upper()
                census_dataset_label = census_dataset[6:]
            elif census_dataset[:1] == 'b':
                census_dataset_code = census_dataset[:6].upper()
                census_dataset_label = census_dataset[7:]
            
            df_census_dataset = pd.read_csv(f'data/generated_data/df_06_{census_dataset_label}_' + year + '_' + office.replace('.', '').replace(' ', '_') + '.csv')
            df_census_dataset.rename(columns={f'geoid_{census_dataset_label}': 'geoidfq_tract'}, inplace=True)
            
            census_dataset_dfs.append(df_census_dataset)
        
        print(f'Merging precinct and census data')
        
        dfs = [df_precincts]
        dfs.extend(census_dataset_dfs)
        
        df = reduce(lambda left, right: pd.merge(left, right, on='geoidfq_tract', how='left'), dfs)
        df['standardized_id_num'] = df['standardized_id_num'].astype(str).str.zfill(13)
        
        print(f'Cleaning columns')
        
        drop_cols = [
            "City/Township Description", "District Code", "Election Type", # would add noise
            "Michigan County Code", # Prefer census code, and would need one-hot-encoding with sparsity
            "Office Description", "Precinct Label", "Status Code",  # would add noise
            "County Name", "Precinct Number", "Election Year",  # would add noise
            "total_votes", "registered_voters", "turnout_pct", # don't look into the future
            "nearest_bound_census_tract", "nearest_bound_school_district", "nearest_bound_zipcode",  # would add noise
            "Office Code", "Census County Code", # Would need one-hot-encoding, not too much sparsity though
            "City/Township Code", "Ward Number",  # would add noise
            "county", "office", "electionye", # addressed elsewhere
            "registered_voters_change", # raw population varies too widely
            "dem_votes", "rep_votes", "oth_votes", # don't look into the future
            "dem_share", "rep_share", "oth_share", # don't look into the future
            "dem_votes_change", "rep_votes_change", "oth_votes_change", # raw population varies too widely
            "locale_full", "objectid", "precinct_num", "precinct_wp_id", # would add noise
            "standardized_id", "geometry", "geometry_tract", "geoidfq_tract", "name_tract", # would add noise
            "subdivision_fips", "ward_num", "locale_full", "county_fips", "objectid", "precinct_num", # would add noise
            "nearest_tract", "awater_tract", "aland_tract", "tractce_tract", "shapestarea", "shapestlength", "geoid_tract", # try not using tract geo characteristics
        ]
        
        string_columns = ["standardized_id_num", "partisanship", "partisanship_change"]
        df = df.drop(columns=drop_cols, errors='ignore')
        numeric_df = df.drop(columns=string_columns, errors='ignore')
        string_df = df[string_columns]
        numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')
        numeric_df = numeric_df.fillna(numeric_df.median())
        df = pd.concat([string_df, numeric_df], axis=1)
        
        df['standardized_id_num'] = df['standardized_id_num'].astype(str).str.zfill(13)
        df = df.dropna(subset=['partisanship_change', 'partisanship_change_amount'])
        
        # Define inactive targets by removing active_target from all_targets
        excluded_targets = [item for item in all_targets if item != active_target]

        # Exclude any other features.
        excluded_features = [
            "standardized_id_num", "registered_voters",
            # "dem_share_prev", "rep_share_prev", "oth_share_prev",  # Huge predictor
            # "dem_share_change", "rep_share_change", "oth_share_change",  # Huge predictor
            # "turnout_pct_change",
        ]
        excluded_features.extend(excluded_targets)
        
        # Encode target and *potential* categorical features. 
        label_encoder = LabelEncoder()
        cat_cols = [active_target]
        for col in cat_cols:
            df[col] = df[col].astype(str)
            df[col] = label_encoder.fit_transform(df[col])
        
        X = df.drop(columns=[active_target] + excluded_features, errors='ignore')
        y = df[active_target]

        # Impute, scale, and split
        print(f'Imputing missing features')
        imputer = SimpleImputer(strategy="median", add_indicator=True)
        X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
        X_imputed = imputer.fit_transform(X)
        new_column_names = list(X.columns) + [f"missing_{i}" for i in range(X_imputed.shape[1] - X.shape[1])]
        X = pd.DataFrame(X_imputed, columns=new_column_names)
        y = y.fillna(y.median())
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X = pd.DataFrame(X_scaled, columns=X.columns)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print(f'Measuring feature importance')
        
        # Correlation
        correlations = X.corrwith(y).abs().sort_values(ascending=False)
        
        # Random Forest
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        rf_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
        
        # Lasso
        lasso = LassoCV(cv=5, random_state=42).fit(X_train, y_train)
        lasso_importances = pd.Series(np.abs(lasso.coef_), index=X.columns).sort_values(ascending=False)
        
        # Mutual info
        mi_scores = mutual_info_regression(X_train, y_train)
        mi_importances = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
        
        # SHAP Values via XGBoost
        xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
        xgb_model.fit(X_train, y_train)
        shap_sample = X_train.sample(n=min(500, len(X_train)), random_state=42) # Use only 500 rows for X_train
        explainer = shap.Explainer(xgb_model, shap_sample)
        shap_values = explainer(shap_sample)
        shap_importances = pd.Series(np.abs(shap_values.values).mean(axis=0), index=X.columns).sort_values(ascending=False)
        
        # Feature importance rankings
        feature_rankings = pd.DataFrame({
            "Correlation": correlations,
            "RandomForest": rf_importances,
            "Lasso": lasso_importances,
            "MutualInfo": mi_importances,
            "SHAP": shap_importances
        })
        feature_rankings_ranked = feature_rankings.rank(axis=0, pct=True, method="average", ascending=False)
        feature_rankings["Average_Rank"] = feature_rankings_ranked.mean(axis=1)
        feature_rankings['Feature name'] = feature_rankings.index[:top_n_features_to_rank]
        feature_rankings.sort_values('Average_Rank', ascending=True)
        feature_rankings.to_csv(f'data/generated_data/feature_rankings_{year}_{office.replace('.', '').replace(' ', '_')}.csv', index=None)
        feature_rankings.drop(columns=['Feature name'], inplace=True)

        # Feature importance histograms
        print(f'Plotting feature importances')
        ranking_types = ["Average_Rank", "Correlation", "RandomForest", "Lasso", "MutualInfo", "SHAP"]        
        for ranking_type in ranking_types:
            features = feature_rankings.sort_values(ranking_type, ascending=False)
            plt.figure(figsize=(60, 140))
            plt.subplots_adjust(left=0.35)
            sns.barplot(x=features[ranking_type] / features[ranking_type].max(), y=features.index, orient="h")
            plt.xticks(rotation=90)
            plt.ylabel("Feature")
            plt.xlabel(ranking_type)
            plt.title(f"Top Features for Predicting {active_target}")
            plt.savefig(f'output/figures/Features_Histogram_{ranking_type}_{year}_{office}.png')
            plt.close()
        
        # Feature correlation heatmap
        top_features = feature_rankings.index[:top_n_features_to_rank]
        corr_matrix = X[top_features].corr()
        # np.fill_diagonal(corr_matrix.values, 0)
        
        plt.figure(figsize=(96, 80))
        plt.subplots_adjust(left=0.35)
        plt.subplots_adjust(bottom=0.35)
        sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", fmt=".2f", linewidths=0.5, square=True)
        plt.title(f"Correlation Heatmap of Top {top_n_features_to_rank} Features for Predicting {active_target}")
        plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        plt.savefig(f'output/figures/Features_Correlation_{year}_{office}.png')
        plt.close()
        
        '''
        Individual Feature Predictive Power (Univariate Performance)
        Evaluate each feature’s ability to predict the target variable alone, using:
        
            Univariate Regression for Continuous Targets
                Train a simple regression model (LinearRegression or DecisionTreeRegressor) on each feature separately.
                Measure R² (coefficient of determination) for how well each feature alone explains variation in y.
        
            Univariate Classification for Categorical Targets
                Train a simple classifier (DecisionTreeClassifier or LogisticRegression) using only one feature at a time.
                Measure accuracy, AUC (for ordinal relationships), or F1-score.
        '''
        print(f'Ranking feature importance')
        feature_performance = []
        is_continuous = y.dtype.kind in 'fc' # float or continuous
        for feature in X.columns:
            X_feature = X[[feature]]
            X_train_feat, X_test_feat, y_train_feat, y_test_feat = train_test_split(X_feature, y, test_size=0.2, random_state=42)
            if is_continuous:  # Regression
                model = LinearRegression()
                model.fit(X_train_feat, y_train_feat)
                y_pred = model.predict(X_test_feat)
                score = r2_score(y_test_feat, y_pred)  # R² for regression
                metric = "R² Score"
            else:  # Classification
                model = LogisticRegression(max_iter=200)  # or DecisionTreeClassifier() if needed
                model.fit(X_train_feat, y_train_feat)
                y_pred = model.predict(X_test_feat)
                score = accuracy_score(y_test_feat, y_pred)  # Accuracy for classification
                metric = "Accuracy"
            feature_performance.append({"Feature": feature, metric: score})
        feature_performance_df = pd.DataFrame(feature_performance).sort_values(by=metric, ascending=False)
        feature_performance_df.to_csv(f'data/generated_data/feature_performance_{year}_{office.replace('.', '').replace(' ', '_')}.csv', index=None)

        '''
        Drop-Column Feature Importance (Permutation Importance)
            Train a full model with all features.
            Then remove one feature at a time, retrain the model, and measure the drop in accuracy.
            A bigger accuracy drop means that feature is highly predictive.
        '''
        print(f'Measuring permutation (random forest regr.) performance')
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        perm_importance = permutation_importance(model, X_test, y_test, scoring="r2")
        perm_importance_df = pd.DataFrame({
            "Feature": X.columns,
            "Importance": perm_importance.importances_mean
        }).sort_values(by="Importance", ascending=False)
        perm_importance_df.to_csv(f'data/generated_data/permutation_important_{year}_{office.replace('.', '').replace(' ', '_')}.csv', index=None)

        '''
        SHAP Values for Predictive Power
            SHAP values tell how much each feature influences a model’s predictions on average.
            Higher SHAP values mean a feature contributes strongly to predictions.
        '''
        print(f'Measuring SHAP performance')
        model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
        model.fit(X_train, y_train)
        explainer = shap.Explainer(model, X_test)
        shap_values = explainer(X_test)
        shap_importance = pd.DataFrame({
            "Feature": X.columns,
            "SHAP Importance": np.abs(shap_values.values).mean(axis=0)
        }).sort_values(by="SHAP Importance", ascending=False)
        shap_importance.to_csv(f'data/generated_data/shap_importance_{year}_{office.replace('.', '').replace(' ', '_')}.csv', index=None)


        ################################################
        # MAKE THE ACTUAL PREDICTIONS
        ################################################
        print(f'Create the predictor')
        
        # Random Forest
        print(f'Train random forest model')
        best_features = rf_importances.index[:top_n_feature_for_preds].tolist()
        valid_features = [feat for feat in best_features if feat in X.columns]
        X_selected = X[valid_features]
        y_encoded = df[active_target]

        # Target column is no longer encoded (unseen labels error)
        assert np.issubdtype(df[active_target].dtype, np.integer)
        
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Predict
        print(f'Make predictions')
        y_pred = model.predict(X_test)

        # Re-convert labels
        all_labels = list(range(len(label_encoder.classes_)))
        target_names = list(label_encoder.classes_)

        print(f'Evaluate predictions')
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy:.4f}")
        print("\nClassification Report:\n", classification_report(y_test, y_pred, labels=all_labels, target_names=target_names))
        
        # Predict each precinct
        df[f"predicted_{active_target}"] = label_encoder.inverse_transform(model.predict(X_selected))
        
        # Reconvert labels
        # df[active_target] = label_encoder.inverse_transform(df[active_target])
        df[f"{active_target}_decoded"] = label_encoder.inverse_transform(df[active_target])

        # Full-dataset accuracy like Excel
        y_all_true = df[active_target]
        y_all_pred = df[f"predicted_{active_target}"]
        overall_accuracy = np.mean(y_all_true == y_all_pred)
        print(f"Full Dataset Accuracy (Excel-style): {overall_accuracy:.4f}")
        
        # Get precinct id, ground truth, and prediction
        df = df[["standardized_id_num", active_target, f"predicted_{active_target}"]]
        df["standardized_id_num"] = df["standardized_id_num"].apply(lambda x: str(x).replace('.0', '').zfill(13))
        df.to_csv(f"data/generated_data/predicted_{active_target}_{year}_{office.replace('.', '').replace(' ', '_')}.csv", index=False)